In [1]:
import numpy as np
from scipy.sparse.linalg import spilu
from scipy.sparse.linalg import lgmres
from scipy.sparse.linalg import LinearOperator
import pickle
from scipy.sparse import csc_matrix
import time

In [2]:
tol = 1e-6
iterations = 0

In [4]:
f = open("a_array", "rb")
matrix = pickle.load(f)
f.close()

f = open("b_array", "rb")
solution = np.load(f)
f.close()

## Create the inverse matrix

While it says in the documentation that the iLU returns an approximate inverse matrix, I wasn't able to find this in the object itself. So I just solve for the identity matrix as has been recommended in some papers dealing with LU decomposition

In [5]:
iLU = spilu(matrix.tocsc(), fill_factor=10)
iLUx = lambda x: iLU.solve(x)
P = LinearOperator(matrix.shape, iLUx)

In [6]:
approx_inverse = iLU.solve(np.identity(32))

### Identity matrix

In [7]:
identity = approx_inverse * matrix

In [8]:
identity

array([[ 1.00000000e+00,  1.20311054e-17,  1.22354995e-16, ...,
        -6.29617492e-12,  9.98613947e-12,  3.47508204e-11],
       [ 3.69160264e-19,  1.00000000e+00, -5.97133690e-17, ...,
         2.82006079e-12, -4.44487910e-12, -1.70997629e-11],
       [ 2.04949593e-17,  7.10491071e-18,  1.00000000e+00, ...,
        -2.81578441e-12,  4.44453339e-12,  1.52196902e-11],
       ...,
       [ 2.12045811e-27, -7.67403889e-27, -4.03896783e-28, ...,
         9.99999989e-01,  1.43497435e-09, -3.99589639e-10],
       [-9.08767763e-28,  4.84676140e-27,  6.05845175e-28, ...,
         1.08444763e-08,  9.99999999e-01,  3.97404581e-10],
       [ 6.46234854e-27, -1.93870456e-26, -2.58493941e-26, ...,
        -6.60969302e-08, -4.49134870e-08,  1.00000001e+00]])

## How good is the inverse matrix

In [9]:
from scipy.sparse.linalg import inv
import random, math

In [10]:
true_inverse = inv(matrix.tocsc())

In [11]:
diff = true_inverse - approx_inverse

In [12]:
av = []

for i in range(1, 400):
    row = random.sample(range(1,32), 1)
    column = random.sample(range(1,32), 1)
    av.append(diff[row, column][0])

mean = np.mean(av)
math.log10(abs(mean))

-7.2464966362977234